In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import random
import datetime as dt

from job_client_graphql import Client, CellDisplay

## Prerequisite
+ Launch JOB server - see [documentation](TBD) section **5-Examples**

In [3]:
pages = {
    'highcharts': 'https://observablehq.com/@oscar6echo/job-example-highcharts',
}

In [4]:
s = Client(server_url='http://localhost:4001/graphql',
           default_channel='channel-1',
           password='mysecret',
           pages=pages,         
           debug=False)
print(s.build_url('highcharts'))
s.show_btn_page('highcharts')

https://observablehq.com/@oscar6echo/job-example-highcharts?host=localhost&port=4001&path=/graphql&ssl=False&channel=channel-1


## Write

In [5]:
def build_hc_opts(data):
    hc_opts = {
        'chart': {'type': 'line','zoomType': 'x'},
        'legend': {'enabled': False},
        'subtitle': {'text': 'Live'},
        'title': {'text': 'Data from Jupyter notebook'},
        'tooltip': {'enabled': True},
        'xAxis': {'type': 'datetime'},
        'yAxis': {'title': {'text': 'Value'}},
        'plotOptions': {'series': {'animation': False}},
        'series': [{'name': 'jupyter-data','data': data}]
    }
    return hc_opts

def rnd():
    return random.randint(1, 10)

def to_unixtime(x):
    return x.timestamp()*1000

In [6]:
N = 30
data = []
now = dt.datetime.now()
for i in range (N):
    t = now + dt.timedelta(seconds=-N+i)
    data.append([to_unixtime(t), rnd()])

def update(x=None):
    if not x:
        x = rnd()
    print(x)
    t = dt.datetime.now()
    data.append([to_unixtime(t), x])


In [10]:
update()
hc_opts = build_hc_opts(data)
s.publish_write('hcOpts', json.dumps(hc_opts))

3


True

## Subscribe

### DISPLAY

In [11]:
out = CellDisplay(name='subscribe-status', max_lines=8)

def show(channel, name, value):
    if name != 'min-max': return
    ts = dt.datetime.now().strftime('%H:%M:%S.%f')[:-4]
    s = f'received on {channel}: {name} --> {value} ({ts})'
    out.update(s, kind='text', append=True)


received on channel-1: min-max --> {"max":1589743218395.565,"min":1589743209493.5527} (21:20:45.86)
received on channel-1: min-max --> {"max":1589743218010.96,"min":1589743213010.96} (21:20:47.69)
received on channel-1: min-max --> {"max":1589743235918.4734,"min":1589743198108.3474} (21:20:49.35)
received on channel-1: min-max --> {'max': 1589743235918.4734, 'min': 1589743198108.3474} (21:20:56.40)

In [12]:
out.display()


received on channel-1: min-max --> {"max":1589743218395.565,"min":1589743209493.5527} (21:20:45.86)
received on channel-1: min-max --> {"max":1589743218010.96,"min":1589743213010.96} (21:20:47.69)
received on channel-1: min-max --> {"max":1589743235918.4734,"min":1589743198108.3474} (21:20:49.35)
received on channel-1: min-max --> {'max': 1589743235918.4734, 'min': 1589743198108.3474} (21:20:56.40)

### START

In [13]:
ref = s.subscribe(action=show, test=lambda x: x=='min-max')

### STOP

In [ ]:
s.subscribe_stop(ref)

In [ ]:
out.clear()

## Read

In [14]:
name = 'min-max'
s.read(name)

{'max': 1589743235918.4734, 'min': 1589743198108.3474}

## Read w/ Histo

In [15]:
name = 'min-max'
s.read_histo(name, 4, timestamp=True)

[]

## Store

Client.store contains the latest values read

In [16]:
s.store

{'channel-1': {'min-max': {'max': 1589743235918.4734,
   'min': 1589743198108.3474}}}

In [ ]:
s.publish_write('hcOpts', None, add_histo=False)